In [1]:
from Libreria.cargar_configbd import cargar_configbd
from Libreria.matriz_de_confusion import matriz_de_confusion

import pandas as pd
import joblib

import xgboost as xgb

In [2]:
# Conectar a la base de datos PostgreSQL
conn = cargar_configbd.conectar_base_datos('conf_bd.txt')
# Crear un cursor para ejecutar consultas
cur = conn.cursor()

In [3]:
modelo_cargado = joblib.load('Modelos/complete_model_rf9992_sept_biciabr.joblib')

# Acceder a los objetos individuales
number_of_features = modelo_cargado['number_of_features']
label_encoder = modelo_cargado['label_encoder']
classifier = modelo_cargado['classifier']
selected_features = modelo_cargado['selected_features']

print("Objetos cargados desde 'complete_model.joblib'")

Objetos cargados desde 'complete_model.joblib'


In [ ]:
# Clasificar dataset restante

In [4]:
def prediccion(modelo, df_new, label_encoder):
    if isinstance(modelo, xgb.Booster):
        # Convertir DataFrame a DMatrix si el modelo es XGBoost
        dmatrix_new = xgb.DMatrix(data=df_new)
        # Hacer predicciones con el nuevo conjunto de datos
        y_pred_new_indices = modelo.predict(dmatrix_new)
        # Invertir la codificación de las etiquetas
        y_pred_new = label_encoder.inverse_transform(y_pred_new_indices.astype(int))
    else:
        # Hacer predicciones directamente con el DataFrame si es otro tipo de modelo
        y_pred_new = modelo.predict(df_new)

    return y_pred_new

In [5]:
meses = ['sept2023']

In [6]:
# Hacer las predicciones
for mes in meses:

    # Crear tablas para guardar etiquetas
    for modo in ['CAMINATA', 'BICICLETA', 'BUS', 'AUTOMOVIL']:

        # Crear tabla para insertar datos corregidos
        consultas_sql = [f"DROP TABLE IF EXISTS inferencia_segmentos_{modo}_{mes};",
            f"""CREATE TABLE IF NOT EXISTS inferencia_segmentos_{modo}_{mes}
                (
                    tripid integer,
                    segid integer,
                    uid character varying(50)
                );
            """]

        for consulta_sql in consultas_sql:
            cur.execute(consulta_sql)
        # Confirma los cambios en la base de datos
        conn.commit()


    cur.execute(f'''SELECT DISTINCT tripid, segid, uid 
                    FROM segmentos_viajes_{mes};
                ''')
    # Obtener los segmentos del mes
    rows = cur.fetchall()
    id_segmentos = rows.copy()

    print(f'----{mes.upper()}----')
    print(f"n_viajes: {len(id_segmentos)}")
    prog_anterior = 1 # En uno aun no precesa nada
    id_segmentos = id_segmentos[prog_anterior - 1:]
    prog = prog_anterior - 1
    for id_segmento in id_segmentos:
        prog += 1
        print(f"Progreso: {prog}", end="\r")   

        # Obtener los puntos del viaje
        cur.execute(f'''SELECT * FROM estadisticos_segmentos_{mes} 
                    where tripid = '{id_segmento[0]}' and segid = '{id_segmento[1]}' and uid = '{id_segmento[2]}'; ''')
        #print(f"SELECT * FROM estadisticos_viajes_sept2023 where tripid = '{id_segmento[0]}' and segid = '{id_segmento[1]}' and uid = '{id_segmento[2]}';")
        df_new = pd.DataFrame(cur.fetchall().copy(), columns=[desc[0] for desc in cur.description])

        pred = prediccion(classifier, df_new[selected_features[:number_of_features]], label_encoder)
        #pred = prediccion(classifier, normalize_data(df_new[selected_features[:number_of_features]]), label_encoder)
        #print(pred[0])

        consulta_sql = """
        INSERT INTO {} VALUES (
            %s, %s, %s
        )
        """.format(f'inferencia_segmentos_{pred[0]}_{mes}')

        # Ejecuta la consulta SQL con los valores de la fila actual
        cur.execute(consulta_sql, id_segmento)
        conn.commit() # Hace commit cuando inserta todo el viaje'''

----SEPT2023----
n_viajes: 35864


In [7]:
infered_data_mes = []
for mes in meses:
    print(mes.upper())

    modos = ['CAMINATA', 'BICICLETA', 'BUS', 'AUTOMOVIL']

    # inferencia_segmentos tienen todos los segmentos clasificados por el modelo supervisado final
    dfs = []

    for modo in modos: 
        
        consulta_sql = f'''
                        SELECT o.*
                            FROM public_new.estadisticos_segmentos_{mes} o
                            JOIN public_new.inferencia_segmentos_{modo}_{mes} i 
                            ON o.tripid = i.tripid
                            AND o.segid = i.segid
                            AND o.uid = i.uid;
                        '''
        
        cur.execute(consulta_sql)
        df = pd.DataFrame(cur.fetchall().copy(), columns=[desc[0] for desc in cur.description])
        df['etiqueta'] = modo

        dfs.append(df)

    infered_data = pd.concat(dfs, ignore_index=True)
    num_elementos_por_grupo = infered_data.groupby('etiqueta').size()
    print(num_elementos_por_grupo)

    infered_data_mes.append(infered_data)
    print("")

print("TOTAL")
df_infered_data_mes = pd.concat(infered_data_mes, ignore_index=True)
num_elementos_por_grupo = df_infered_data_mes.groupby('etiqueta').size()
print(num_elementos_por_grupo)

SEPT2023
etiqueta
AUTOMOVIL     7887
BICICLETA     2549
BUS          10097
CAMINATA     15331
dtype: int64

TOTAL
etiqueta
AUTOMOVIL     7887
BICICLETA     2549
BUS          10097
CAMINATA     15331
dtype: int64


In [27]:

for mes in ['sept2023']:
  # -----Agregar columnas para caracteristicas nuevas
  consulta_sql = f'''DROP TABLE IF EXISTS datos_uda_inferencia_{mes};
                    CREATE TABLE datos_uda_inferencia_{mes} (
                        tripid INTEGER,
                        segid INTEGER,
                        uid VARCHAR(255),
                        p25_speed FLOAT, 
                        p75_speed FLOAT,
                        max_speed FLOAT,
                        min_speed FLOAT,
                        mean_speed FLOAT,
                        median_speed FLOAT,
                        var_speed FLOAT,
                        p25_aceleration FLOAT, 
                        p75_aceleration FLOAT,
                        max_aceleration FLOAT,
                        min_aceleration FLOAT,
                        mean_aceleration FLOAT,
                        median_aceleration FLOAT,
                        var_aceleration FLOAT,
                        p25_bearing FLOAT, 
                        p75_bearing FLOAT,
                        max_bearing FLOAT,
                        min_bearing FLOAT,
                        mean_bearing FLOAT,
                        median_bearing FLOAT,
                        var_bearing FLOAT,
                        p25_time_stop FLOAT, 
                        p75_time_stop FLOAT,
                        max_time_stop FLOAT,
                        min_time_stop FLOAT,
                        mean_time_stop FLOAT,
                        median_time_stop FLOAT,
                        var_time_stop FLOAT,
                        stops INTEGER,
                        etiqueta VARCHAR(50)
                    );
                  '''
  cur.execute(consulta_sql)
  cur.execute(f"commit")

In [28]:
for mes, infered_data in zip(['sept2023'], infered_data_mes):

    print(f'----{mes.upper()}----')
    print(f"n_viajes_inferidos: {len(infered_data)}")
    prog = 0
    for idx, row in infered_data.iterrows():
        prog += 1
        print(f"Progreso: {prog}", end="\r")  
        
        consulta_sql = """
        INSERT INTO {} VALUES (
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
            %s, %s, %s
        )
        """.format(f'datos_uda_inferencia_{mes}')

        cur.execute(consulta_sql, tuple(row))
        conn.commit()

----SEPT2023----
n_viajes_inferidos: 38943
